# Chp6. Decision Analysis

- __발표자 : 임진주__
- __발표일 : 2017. 9. 2(금)__
---

## The Price is Right
The Price is Right는 미국 게임 쇼입니다. 목표는 경품의 가격을 추측하는 것입니다. 실제 가격을 넘지 않고 쇼케이스의 실제 가격에 가장 근접한 참가자가 수상합니다.  
Nathaniel의 쇼케이스에는 식기 세척기, 와인 캐비닛, 노트북 컴퓨터, 그리고 자동차가 포함되어있었고, 그는 \$26,000를 입찰했습니다. Letia의 쇼케이스에는 핀볼 머신, 비디오 아케이드 게임, 당구대 및 바하마 크루즈가 포함되어 있습니다. 그녀는 \$21,500을 입찰합니다.  
Nathaniel의 쇼케이스의 실제 가격은 \$25,347입니다. 그의 입찰가가 너무 높아서 졌습니다.  
Letia의 쇼케이스의 실제 가격은 \$21,578입니다. 실제 가격과의 차이는 \$78에 불과했기 때문에 그녀는 쇼케이스에서 이겼습니다.  
Bayesian의 경우,이 시나리오는 다음과 같은 몇 가지 질문을 제시합니다.  
1. 상품을보기 전에 참가자는 쇼케이스 가격에 대해 어떤 prior belief을 가지고 있어야합니까?
2. 상품을 본 후 참가자는 어떻게 그 belief을 업데이트해야합니까?
3. 사후 분포를 기반으로 참가자는 무엇을 입찰해야합니까?
세 번째 질문은 베이즈 분석의 공통적인 이용: 의사 결정 분석을 보여줍니다.
사후 분포가 주어지면 참가자의 기대 수익을 최대화하는 입찰가를 선택할 수 있습니다.

## The Prior
<img src="figures/1.png" width=500><img/>
[그림 6-1]  
이 쇼케이스의 가격 분포를 보여줍니다. 두 쇼케이스의 일반적인 가치는 약 \$28,000이지만 첫 번째 쇼케이스는 두 번째 mode가 \$50,000이며, 두 번째 쇼케이스는 때때로 \$70,000 이상 가치가 있습니다. 이러한 분포는 실제 데이터를 기반으로 하지만 ** 가우스 커널 밀도 추정(Gaussian kernel density estimation; KDE)**에 의해 부드럽게 처리되었습니다.

## Probability density functions

주어진 값 x에 대해이 함수는 확률 밀도를 계산합니다. 밀도는 밀도가 높을수록 값이 더 있음을 의미하는 확률 덩어리와 유사합니다. 그러나 밀도는 확률이 아닙니다. 밀도는 0 또는 임의의 양수 값일 수 있습니다. 0과 1 사이의 확률과 같이 제한되지 않습니다. 연속 범위에서 밀도를 적분하면 그 결과는 확률입니다. 그러나 이 책의 응용 프로그램에서는 거의 그렇게 할 필요가 없습니다. 대신에 likelihood function의 일부로 확률 밀도를 주로 사용합니다.
<img src="figures/0.png" width=250><img/>
[식 6-1] gaussian pdf

## Representing PDFs
Python에서 PDF를 표현하기 위해 thinkbayes.py는 Pdf라는 클래스를 제공합니다. Pdf는 abstract type입니다. 즉, Pdf가 가져야 할 인터페이스를 정의하지만 완전한 구현을 제공하지는 않습니다. Pdf 인터페이스에는 Density와 MakePmf라는 두 가지 method이 포함됩니다.

In [ ]:
class Pdf(object):
    def Density(self, x):
        raise UnimplementedMethodException()
        
    def MakePmf(self, xs):
        pmf = Pmf()
        for x in xs:
            pmf.Set(x, self.Density(x))
            pmf.Normalize()
        return pmf

Density는 값 x를 취해 해당 밀도를 반환합니다. MakePmf는 PDF에 대한 이산적인 근사값을 만듭니다.
Pdf는 MakePmf의 구현을 제공하지만 하위 클래스가 제공해야하는 Density는 제공하지 않습니다.
concrete type은, abstract type을 확장해, 비어있는 메소드의 구현을 제공하는 아이 클래스입니다. 예를 들어, GaussianPdf는 Pdf를 확장하고 Density를 제공합니다.

In [ ]:
class GaussianPdf(Pdf):
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def Density(self, x):
        return scipy.stats.norm.pdf(x, self.mu, self.sigma)

Density는 scipy.stats의 함수를 사용하여 가우스 PDF를 계산합니다. 가우시안 분포를 "normal" 분포라고도 부르기 때문에이 함수는 norm.pdf라고 합니다. Gaussian PDF는 간단한 수학 함수에 의해 정의되므로 쉽게 계산할 수 있습니다. 실제 세계의 많은 것들이 근사적으로 가우스 분포를 가지므로 유용합니다. 그러나 실제 데이터를 사용하면 분포가 가우스 또는 다른 간단한 수학 함수라는 보장은 없습니다. 이 경우 샘플을 사용하여 전체 인구의 PDF를 추정 할 수 있습니다.  

예를 들어, Price Is Right 데이터에는 첫 번째 쇼케이스에 대해 313 개의 가격이 있습니다. 우리는 이러한 값들을 가능한 모든 경품 가격의 샘플로 생각할 수 있습니다. 이 샘플에서는 28801과 28867 사이에 값이 나타나지 않지만 이러한 값을 사용할 수 없다고 생각할 이유는 없습니다. 우리의 배경 정보를 바탕으로이 범위의 모든 값이 똑같이 나타날 것으로 기대합니다. 즉, PDF가 상당히 매끄럽기를 기대합니다.
KDE (Kernel density estimation)는 샘플을 가져 와서 데이터에 맞는 적절하게 부드러운 PDF를 찾는 알고리즘입니다. https://en.wikipedia.org/wiki/Kernel_density_estimation

In [ ]:
class EstimatedPdf(Pdf):
    def __init__(self, sample):
        self.kde = scipy.stats.gaussian_kde(sample)
        
    def Density(self, x):
        return self.kde.evaluate(x)

In [ ]:
prices = ReadData()
pdf = thinkbayes.EstimatedPdf(prices)
low, high = 0, 75000
n = 101
xs = numpy.linspace(low, high, n)
pmf = pdf.MakePmf(xs)

pdf는 KDE에서 추정 한 Pdf 객체입니다. pmf는 Pmf 객체로, 균일하게 간격을 둔 값의 시퀀스에서 밀도를 계산하여 Pdf를 근사합니다.
linspace는 "linear space"의 약자로, 범위, low, high와 점의 수는 n을 입력하면, n개의 요소가 모두 low와 high 사이에 균등하게 간격을 둔 새로운 numpy array가 반환됩니다.

## Modeling the contestants
이 prior을 업데이트하려면 다음 질문에 답해야합니다.
1. 우리는 어떤 데이터를 고려해야하고 어떻게 계량화해야합니까?
2. likelihood function을 계산할 수 있습니까? 즉, 가격의 각 가설 값에 대해 데이터의 조건부 확률을 계산할 수 있습니까?

이 질문에 답하기 위해, 참가자를 알려진 error characteristic을 가진 가격 추측 도구로 모델링 할 것입니다. 즉, 참가자가 상품을 보게되면 상금이 쇼케이스의 일부라는 사실을 고려하지 않고 각 상품의 가격을 추측하고 가격을 합산합니다. 이를 total guess라고합시다.  
이 모델에서 우리가 대답해야하는 질문은 "실제 가격이 price이라면 참가자의 guess는 얼마 일까?"입니다.  
혹은 *error = price - guess* 라고 정의하고, "참가자의 estimate이 error를 낼 가능성은 어느 정도입니까?"라고 물을 수 있습니다.

<img src="figures/2.png" width=500><img/>
[그림 6-2] Cumulative distribution (CDF) of the difference between the contestant’s
bid and the actual price.

그림 6-2는 참가자의 입찰가와 쇼케이스의 실제 가격의 차이 인 diff의 누적 분포를 보여줍니다.  
diff의 정의는 *diff = price - bid*입니다.  
diff가 음수이면 입찰가가 너무 높다는 것입니다. 어쨌든, 우리는 이 분포를 사용하여 참가자가 오버해서 입찰할 확률을 계산할 수 있습니다. 첫 번째 참가자는 25% 정도 오버해서 입찰합니다. 두 번째 참가자는 29%를 오버해서 입찰합니다. 또한 입찰가가 편향된 것을 볼 수 있습니다.

마지막으로 이 분포를 사용하여 참가자의 guess의 신뢰도를 추정 할 수 있습니다. 이 단계는 참가자의 guess를 실제로 알지 못하기 때문에 약간 까다롭습니다. 우리는 그들이 입찰한 가격만 알고 있죠.  
그래서 우리는 몇 가지 가정을 해야 하는데, 특히, error의 분포가 평균 0이고 diff와 동일한 분산이라고 가정합니다.

In [ ]:
class Player(object):
    def __init__(self, prices, bids, diffs):
        self.pdf_price = thinkbayes.EstimatedPdf(prices)
        self.cdf_diff = thinkbayes.MakeCdfFromList(diffs)
        
        mu = 0
        sigma = numpy.std(diffs)
        self.pdf_error = thinkbayes.GaussianPdf(mu, sigma)

pdf_error는 error의 분포를 특성화한 PDF입니다. (error = price - guess)  
다시, diff의 분산을 사용하여 error의 분산을 추정합니다. 이 견적은 참가자의 입찰가가 때로는 전략적이기 때문에 완벽하지 않습니다. 예를 들어, 플레이어 2가 플레이어 1이 과도하게 생각한다고 생각하면 플레이어 2는 매우 낮은 입찰가를 적용 할 수 있습니다. 이 경우 diff는 error를 제대로 반영하지 않습니다. 이런 경우가 많이 발생하면 diff에서 관찰된 분산이 error의 분산을 과대 평가할 수 있습니다. 그럼에도 불구하고 나는 이런 모델링이 일리가 있는 결정이라고 생각합니다.

## Likelihood

In [ ]:
class Price(thinkbayes.Suite):
    def __init__(self, pmf, player):
        thinkbayes.Suite.__init__(self, pmf)
        self.player = player

    def Likelihood(self, data, hypo):
        price = hypo
        guess = data
        error = price - guess
        like = self.player.ErrorDensity(error)
        return like

pmf는 사전 분포를 나타내고 player는 이전 섹션에서 설명한대로 Player 개체입니다.  
hypo는 쇼케이스의 가설적인 가격입니다. 데이터는 가격에서 참가자의 가장 좋은 guess입니다. error는 그 차이이며, like는 가설을 가정 할 때 데이터의 likelihood입니다.  
ErrorDensity는 Player 객체에 정의되어있습니다.

In [ ]:
# class Player:
    def ErrorDensity(self, error):
        return self.pdf_error.Density(error)

ErrorDensity는 주어진 error 값에서 pdf_error를 평가하여 작동합니다. 결과는 density이므로 실제 확률은 아닙니다. 그러나 Likelihood는 확률을 계산할 필요가 없다는 것을 기억하십시오. 단지 확률에 비례하는 어떤 것만 계산하면 됩니다. 나누는 상수가 모든 likelihood에 대해 동일하다면, 우리가 사후 분포를 정규화 할 때 그것은 상쇄되니까요.

## Update

In [ ]:
# class Player
    def MakeBeliefs(self, guess):
        pmf = self.PmfPrice()
        self.prior = Price(pmf, self)
        self.posterior = self.prior.Copy()
        self.posterior.Update(guess)

PmfPrice는 가격의 PDF에 대한 이산 근사값을 생성합니다. 이 근사값을 사용하여 prior를 구성합니다.  
PmfPrice는 값의 sequence에서 pdf_price를 계산하는 MakePmf를 사용합니다

In [ ]:
# class Player
    n = 101
    price_xs = numpy.linspace(0, 75000, n)
    
    def PmfPrice(self):
        return self.pdf_price.MakePmf(self.price_xs)

posterior를 구성하기 위해 이전의 복사본을 만들고 Update를 호출합니다.이 Update는 각 가설에 대해 likelihood를 호출하고 prior와 likelihood를 곱한 다음 재정규화합니다. 
<img src="figures/3.png" width=500><img/>
[그림 6-3] Prior and posterior distributions for Player 1, based on a best guess of
$20,000.

원래 시나리오로 돌아가 봅시다. 플레이어 1이라고 가정하고 쇼케이스를 보았을 때 총 상품 가격에 대한 guess는 \$20,000입니다. 그림 6-3은 실제 가격에 대한 전후의 belief를 보여줍니다. guess는 prior 범위의 하단에 있기 때문에 posterior가 왼쪽으로 이동합니다.  
이 결과는 의미가 있습니다. prior의 가장 가능성 있는 값은 \$27,750이고 가장 좋은 guess는 \$20,000이며 posterior의 평균은 그 사이 어딘가, \$25,096입니다.  
다른 면에서 보면 이 결과가 이상합니다. 가격이 \$20,000라고 guess하면 가격이 \$24,000라고 believe해야 하기 때문이죠.  
이 역설을 해결하려면 두 가지 정보, 과거 쇼케이스에 대한 기록 데이터 및 당신이 보는 경품에 대한 guess를 결합해야합니다.  
우리는 과거 데이터를 prior로 간주하고 guess에 따라 업데이트하고 있지만, guess를 prior로 놓고 과거 데이터를 기반으로 업데이트 할 수도 있습니다.

In [ ]:
class GainCalculator(object):
    def __init__(self, player, opponent):
        self.player = player
        self.opponent = opponent

player와 opponent는 Player 개체입니다.  
GainCalculator는 ExpectedGains를 제공합니다. ExpectedGains는 일련의 입찰가와 각 입찰가의 기대 이익을 계산합니다.

In [ ]:
def ExpectedGains(self, low=0, high=75000, n=101):
    bids = numpy.linspace(low, high, n)
    gains = [self.ExpectedGain(bid) for bid in bids]
    return bids, gains

low와 high로 가능한 입찰가 범위를 지정합니다. n은 시도할 입찰가의 수입니다.
ExpectedGains는 ExpectedGain을 호출합니다. ExpectedGain은 주어진 입찰에 대한 기대 이익을 계산합니다.

In [ ]:
def ExpectedGain(self, bid):
    suite = self.player.posterior
    total = 0
    for price, prob in sorted(suite.Items()):
        gain = self.Gain(bid, price)
        total += prob * gain
    return total

ExpectedGain은 posterior의 값을 통해 반복문을 수행하고 쇼케이스의 실제 가격이 주어지면 각 입찰에 대한 이득을 계산합니다. 해당 확률로 각 이득을 가중치하고 합계를 반환합니다.  
ExpectedGain은 Gain을 호출합니다. Gain은 입찰가와 실제 가격을 받아 기대 이익을 반환합니다.

In [ ]:
def Gain(self, bid, price):
    if bid > price:
        return 0
    diff = price - bid
    prob = self.ProbWin(diff)
    if diff <= 250:
        return 2 * price * prob
    else:
        return price * prob

당신이 오버해서 입찰하면, 당신은 아무것도 얻지 못합니다. 그렇지 않는다면 우리는 당신이 이길 확률을 결정 짓는 입찰가와 가격의 차이를 계산합니다.  
diff가 \$250 미만이면 양쪽 쇼케이스에서 모두 승리합니다. 단순하게 하기 위해 두 쇼케이스 모두 동일한 가격을 유지한다고 가정합니다. 이 결과는 드물기 때문에 큰 차이는 없습니다.  
마지막으로, diff를 기반으로 이길 확률을 계산해야합니다.

In [ ]:
def ProbWin(self, diff):
    prob = (self.opponent.ProbOverbid() +
    self.opponent.ProbWorseThan(diff))
    return prob

상대가 오버 입찰하면 이기게됩니다. 그렇지 않으면, 당신은 당신의 상대가 diff 이상으로 틀리기를 바랍니다. Player는 두 확률을 모두 계산하는 방법을 제공합니다.

In [ ]:
# class Player:
    def ProbOverbid(self):
        return self.cdf_diff.Prob(-1)

<img src="figures/4.png" width=500><img/>
[그림 6-4] Expected gain versus bid in a scenario where Player 1’s best guess is \$20,000
and Player 2’s best guess is \$40,000.

In [ ]:
    def ProbWorseThan(self, diff):
        return 1 - self.cdf_diff.Prob(diff)

이 코드는 상대방의 관점에서 보는 계산법이라 헷갈릴 수 있습니다. 상대방은 '내가 과장 할 확률은 얼마일까?'와 '내 입찰가가 차액을 넘을 확률은 얼마일까?'를 계산합니다.  
두 답변 모두 diff의 CDF를 기반으로합니다. 상대의 diff가 -1보다 작거나 같으면 승리하고, 그렇지 않으면 잃게됩니다.  
마지막으로 최적의 입찰가를 계산하는 코드는 다음과 같습니다:

In [ ]:
# class Player:
    def OptimalBid(self, guess, opponent):
        self.MakeBeliefs(guess)
        calc = GainCalculator(self, opponent)
        bids, gains = calc.ExpectedGains()
        gain, bid = max(zip(gains, bids))
        return bid, gain

guess와 상대를 감안할 때 OptimalBid는 사후 분포를 계산하고 GainCalculator를 인스턴스화하며 일정 범위의 예상 이익을 계산하고 최적의 입찰 및 기대 이익을 반환합니다. 휴!  
그림 6-4는 플레이어 1이 가장 좋은 시나리오를 기반으로 두 플레이어의 결과를 보여줍니다.
추측은 \$20,000이고 플레이어 2의 guess는 \$40,000입니다.  
플레이어 1의 경우 최적 입찰가는 \$21,000이며, 예상 수익은 거의 \$16,700입니다. 이는 최적의 입찰가가 실제로 참가자의 가장 좋은 guess보다 높은 경우(드문 경우임)입니다.  
플레이어 2의 경우 최적의 입찰가는 \$31,500이며 예상 수익은 \$19,400입니다. 이것은 최적의 입찰가가 가장 좋은 guess보다 적은 전형적인 경우입니다.

## Discussion

이 예에서는 사후 분포를 사용하여 최적의 입찰가를 계산합니다. 주어진 입찰가의 수익률은 비대칭이고 불연속적이므로 (과도하게 입찰하면 잃게 됨) 분석적으로이 문제를 해결하기가 어려울 수 있습니다. 그러나 계산을 하는 것은 비교적 간단합니다.  
Bayesian 사고를 처음 접한 사람들은 종종 평균 또는 최대 우도 추정치를 계산하여 사후 분포를 요약하려고 합니다. 이러한 요약은 유용 할 수 있지만 그게 전부라면, 처음에는 베이지안 방법이 필요하지 않을 것입니다.  
베이지안 방법은 사후 분포를 분석의 다음 단계로 가져 와서 이 장에서했던 것처럼 일종의 의사 결정을 수행하거나 다음 장에서 볼 수있는 일종의 예측을 수행 할 때 가장 유용합니다.